In [ ]:
import os
len(os.listdir('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/train'))

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install rouge

In [ ]:
import re 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
 
stop_words = set(stopwords.words('english'))

import spacy

from collections import Counter

import torch

# metrics
from difflib import SequenceMatcher
from rouge import Rouge

In [ ]:
# Remove boundaries [PERSONx], (PERSONy) etc. to PERSONx, PERSONy
import re
 
def rem_boundries(wordList):
 
  processed_list = []
 
  reg_Ex = [r"\(", r"\)", r"\[", r"]"]
 
  for word in wordList:
    for exp in reg_Ex:
      word = re.sub(exp, '', word)
    processed_list.append(word)
 
  return processed_list

In [ ]:
def brac_match(reg_ex, TextList):
  
  processed_list = []
 
  for line in TextList:
    for exp in reg_ex:
      result = re.findall(exp, line)
      if len(result) > 0:
        for word in result:
          if word not in processed_list:
            processed_list.append(word)
 
  return processed_list

In [ ]:
def get_brac_cnt(Textlist):
  
  Reg_Ex = [r"\(PERSON\d+\)", r"\[PERSON\d+\]"] 
  Reg_Ex_ = [r"\[ORGANIZATION\d+\]", r"\[PROJECT\d+\]"]
  
  List_A = brac_match(Reg_Ex, Textlist)
  List_A =  rem_boundries(List_A)
 
  List_B = brac_match(Reg_Ex_, Textlist)
  List_B =  rem_boundries(List_B)
 
  if len(List_B) > len(List_A):
    return List_B
 
  return list(set(List_A))
 
# x, y = get_brac_cnt(minutes), get_brac_cnt(transcripts)

In [ ]:
def chk_person_match(meet_br_cnt, trans_br_cnt):
 
  similar_cnt = []
 
  for word in meet_br_cnt:
    if word in trans_br_cnt:
      similar_cnt.append(word)
 
  return similar_cnt
 
# c = chk_person_match(x, y)
# c, len(c), len(x)

In [ ]:
def remove_cntsqbr(text_list):
  
  new_list = []
 
  for word in text_list:
    new_list.append(re.sub(r"( +)\[(.*?)\]", '', word))
 
  return new_list

In [ ]:
# toknize text
def remove_punctuation(text):
 
  tokenizer = RegexpTokenizer(r"\w+")
 
  return ' '.join(tokenizer.tokenize(text))

In [ ]:
# remove stopwords from the text
def remove_stopwords(stop_words, text):
 
  wordsList = word_tokenize(text)
 
  wordsList = [w for w in wordsList if w not in stop_words]
 
  return wordsList

In [ ]:
# add positional tag to word tokens
def add_postag(word_tokens):
  
  tagged = nltk.pos_tag(word_tokens)
 
  return tagged

In [ ]:
def add_postag_spacy(text):

  tagged = []

  nlp = spacy.load("en_core_web_sm")

  word_tokens = nlp(text)

  for token in word_tokens:
    tagged.append((token, token.pos_))

  return tagged

In [ ]:
def get_NOUNS_spacy(tagged_tokens):

  Extracted_Nouns = []

  Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NOUN']

  return Extracted_Nouns

# get_NOUNS_spacy(tagged)

In [ ]:
def aggregate_nouns_spacy(textList):

  tot_nouns = []

  for text in textList:
    text = remove_punctuation(text)
    tag_list = add_postag_spacy(text)
    nouns = get_NOUNS_spacy(tag_list)
    tot_nouns.extend(nouns)

  tot_nouns = [word for word in tot_nouns if len(word) > 6]

  return tot_nouns

In [ ]:
def get_NNS(tagged_tokens):
 
  Extracted_Nouns = []
    
  # Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NN' or tag == 'NNS']
  Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NN' or tag == 'JJ']
  # Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NNP' or tag == "NN"]
 
  return Extracted_Nouns

In [ ]:
def aggregate_nouns(stop_words, textList):
 
  tot_nouns = []
 
  for text in textList:
    text = remove_punctuation(text)
    word_tokens = remove_stopwords(stop_words, text)
    tag_list = add_postag(word_tokens)
    nouns = get_NNS(tag_list)
    tot_nouns.extend(nouns)
 
  tot_nouns = [word for word in tot_nouns if len(word) > 5]
 
  return tot_nouns
 
# x = aggregate_nouns(stop_words, transcripts)
# z = aggregate_nouns(stop_words, minutes)

In [ ]:
def Counter_Transcripts(transcript):
 
  return Counter(transcript)

In [ ]:
def restrict_counter(Counter_Transcript):
 
    prepro_counter = dict()
 
    for key, value in Counter_Transcript.items():
 
      # if value < 10:
      #   prepro_counter[key] = value
 
      if value > 10:
        prepro_counter[key] = 10
      else:
        prepro_counter[key] = value
 
 
    return prepro_counter

In [ ]:
def get_similar_nouns(minute_nouns, transcript_nouns):
 
  similar_nouns = [w for w in minute_nouns if w in transcript_nouns]
 
  return similar_nouns

In [ ]:
def meet_word_count(minute_nouns, transcript_nouns_Counter):
 
  transcript_Counter = restrict_counter(transcript_nouns_Counter)
 
  trans_nouns_set = list(transcript_Counter.keys())
  trans_nouns_count = list(transcript_Counter.values())
 
  count_p_index = []
 
  for word in minute_nouns:
    for idx, t_word in enumerate(trans_nouns_set):
      if word == t_word:
        count_p_index.append(trans_nouns_count[idx])
 
  return count_p_index

In [ ]:
def calculate_Avg(minute_Count):
 
  c_Tensor = torch.Tensor(minute_Count)
 
  mean = torch.mean(c_Tensor)
 
  return mean.item()

In [ ]:
def concat_nouns(minute_list, transcript_list):
 
  Nounlist_A = list(set(minute_list))
 
  trans_Counter = Counter_Transcripts(transcript_list)
  Trans_dict = restrict_counter(trans_Counter)  
  Nounlist_B = Trans_dict.keys()
 
  minute_str = ' '.join(Nounlist_A)
  transcript_str = ' '.join(Nounlist_B)
 
  return minute_str, transcript_str

In [ ]:
def lowercase(word_list):

  lower_list = []

  for word in word_list:
    lower_list.append(word.lower())

  return lower_list

def apply_stemming(word_list, flag=True):
  
  if flag:
    word_list = list(set(word_list))

  ps = PorterStemmer()  

  stemmed_words = []

  for word in word_list:
    stemmed_words.append(ps.stem(word))

  if flag:
    stemmed_words = list(set(stemmed_words))

  return stemmed_words

In [ ]:
def match_words(Minute_nouns, Transcript_nouns):

  Minute_nouns = list(set(Minute_nouns))  
  Transcript_nouns = list(set(Transcript_nouns))

  Count = 0
  for word in Minute_nouns:
    if word in Transcript_nouns:
      Count += 1

In [ ]:
To_remove_list = ['something', 'meeting', 'transcript', 'arrival', 'system', 'suggestion', 'annotation']

def remove_from(text_list):

  text_list = list(set(lowercase(text_list)))

  sample = []

  for word in text_list:
    if word not in To_remove_list:
      sample.append(word)

  return sample

----
#### Score 1
----

In [ ]:
# Score 1
# aggregate methods to generate score

def cnt_aggregator(minute, transcript, threshold = 0.8):

  minute_cnt = get_brac_cnt(minute)
  if len(minute_cnt) == 0:
    return 0
  else:
    trans_cnt = get_brac_cnt(transcript)
    match = chk_person_match(minute_cnt, trans_cnt)
    m_len = len(match)
    min_len = len(minute_cnt)
    score = len(match)/len(minute_cnt)
    if score > threshold:
      return 1
    else: 
      return 0

----
#### Score2
----

In [ ]:
# Score 2
# get the nouns list for both minutes and Transcript
# apply aggregate function

def overall_aggregator(minute, transcript, stopwords):

  # Remove the content within square brackets 
  minute = remove_cntsqbr(minute)
  transcript = remove_cntsqbr(transcript)  

  # Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
  Minute_nouns = aggregate_nouns(stopwords, minute)
  Transcript_nouns = aggregate_nouns(stopwords, transcript)

  # Apply Counter to Transcripts
  Transcript_nouns_Counter = Counter(Transcript_nouns)

  # check the word count corresponding to similarity of minute nouns in transcript nouns
  word_count = meet_word_count(Minute_nouns, Transcript_nouns_Counter)

  # Calculat Average distinct word count of minutes nouns in transcript nouns
  avg_value = calculate_Avg(word_count)

  return avg_value

# average = overall_aggregator(minutes, transcripts, stop_words)

# overall_aggregator(minutes, transcripts, stop_words)

----
#### Score 3 - 7
----

In [ ]:
# calculate cosine similarity 
def cosine_similarity(X,Y):
  
  X_list = word_tokenize(X) 
  Y_list = word_tokenize(Y)
  sw = stopwords.words('english') 
  l1 =[];l2 =[]
  X_set = {w for w in X_list if not w in sw} 
  Y_set = {w for w in Y_list if not w in sw}


  rvector = X_set.union(Y_set) 
  for w in rvector:
      if w in X_set: l1.append(1) # create a vector
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
  c = 0


  for i in range(len(rvector)):
          c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)
  return cosine

In [ ]:
# Seqeunce matcher function
def sequence_matcher(a, b):
  return SequenceMatcher(None, a, b).ratio()

In [ ]:
# Rouge scores rouge1 and rougel
def Rouge_Score(hypothesis, reference):
  rouge = Rouge()
  scores = rouge.get_scores(hypothesis, reference)
  
  return scores

In [ ]:
# self defined res function
def res(a,b):
    return len(set(a.split(' ')) & set(b.split(' '))) / float(len(set(a.split(' ')) | set(b.split(' ')))) *100

In [ ]:
# aggregate methods and calculate score 3 to 7

def aggregate_concat(minutes, transcripts):
 
  # Remove the content within square brackets 
  minute = remove_cntsqbr(minutes)
  transcript = remove_cntsqbr(transcripts)  
 
  # Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
  Minute_nouns = aggregate_nouns(stop_words, minute)
  Transcript_nouns = aggregate_nouns(stop_words, transcript)
 
  minute_str, transcript_str = concat_nouns(Minute_nouns, Transcript_nouns)
 
  cos = cosine_similarity(minute_str, transcript_str)
  r_1 = Rouge_Score(minute_str, transcript_str)[0]['rouge-1']['f']
  r_l = Rouge_Score(minute_str, transcript_str)[0]['rouge-l']['f']
  seq = sequence_matcher(minute_str, transcript_str)
  reS = res(minute_str, transcript_str)
 
  return cos, r_1, r_l, seq, reS